In [1]:
import torch
import monai
from torch.export import export
from executorch.exir import to_edge

/home/kindersc/Documents/model_conversion/executorch_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
densenet = getattr(monai.networks.nets, 'densenet121')
model = densenet(spatial_dims=2,
                    in_channels=3,
                    out_channels=3,
                    dropout_prob=float(0.2),
                    pretrained=True)

In [4]:
# Load the weights from the file
weights_path = '/home/kindersc/Documents/executorch_example/executorch/notebooks/best_metric_model.pth'
state_dict = torch.load(weights_path, map_location=torch.device('cpu'))

/tmp/ipykernel_21966/579817510.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path, map_location=torch.device('cpu'))


In [5]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [6]:
model.eval()

DenseNet121(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (dropout): Dropout2d(p=0.2, inplace=False)
        )
      )
      (denselayer2): _DenseLayer(
  

In [7]:
# 1. torch.export: Defines the program with the ATen operator set.
aten_dialect = export(model, (torch.ones(1, 3, 256, 256),))

In [8]:
# 2. to_edge: Make optimizations for Edge devices
edge_program = to_edge(aten_dialect)

In [9]:
# 3. to_executorch: Convert the graph to an ExecuTorch program
executorch_program = edge_program.to_executorch()

In [10]:
# 4. Save the compiled .pte program
with open("model_threesix.pte", "wb") as file:
    file.write(executorch_program.buffer)